In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [2]:
OUTFILE = '../data/midterm_data.xlsx'

STARTDATE = '2015-01-01'
ENDDATE = '2025-07-05'

***

In [3]:
TICKS =  ['SPY','AAPL','NVDA','MSFT','GOOGL','AMZN','META','TSLA','AVGO','BRK-B','LLY','SHV']

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [4]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [5]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,63670226,603516960768,The trust seeks to achieve its investment obje...
AAPL,Apple Inc.,EQUITY,USD,39765812,NaN,"Apple Inc. designs, manufactures, and markets ..."
NVDA,NVIDIA Corporation,EQUITY,USD,193633263,NaN,"NVIDIA Corporation, a computing infrastructure..."
MSFT,Microsoft Corporation,EQUITY,USD,16459512,NaN,Microsoft Corporation develops and supports so...
GOOGL,Alphabet Inc.,EQUITY,USD,34282922,NaN,Alphabet Inc. offers various products and plat...
AMZN,"Amazon.com, Inc.",EQUITY,USD,50518307,NaN,"Amazon.com, Inc. engages in the retail sale of..."
META,"Meta Platforms, Inc.",EQUITY,USD,10873880,NaN,"Meta Platforms, Inc. engages in the developmen..."
TSLA,"Tesla, Inc.",EQUITY,USD,79236442,NaN,"Tesla, Inc. designs, develops, manufactures, l..."
AVGO,Broadcom Inc.,EQUITY,USD,14274674,NaN,"Broadcom Inc. designs, develops, and supplies ..."


In [6]:
tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE, auto_adjust=False)['Close']

[*********************100%***********************]  12 of 12 completed


In [7]:
AGG = 'W-FRI'
prices = df.resample(AGG).last().iloc[:-1,:]
prices.index.name = 'date'

# eliminate timezones
prices.index = prices.index.tz_localize(None)

rets = prices.pct_change().dropna()

retsx = rets.subtract(rets['SHV'], axis=0)

In [8]:
info = info.drop(index=['SHV'])
retsx = retsx.drop(columns=['SHV'])

***

## Export Data

In [9]:
with pd.ExcelWriter(OUTFILE) as writer:    
    info.to_excel(writer, sheet_name='info')
    retsx.to_excel(writer, sheet_name='excess returns')